In [ ]:
import os
import json
from typing import List, Dict

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
OPENAI_API_KEY = ""

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
    temperature=0.0,
    max_tokens=1000,
    top_p=0.3,
    frequency_penalty=0.1
)


In [ ]:
# # 한글 변환 
# Label1 = {  
#     "건강웰빙": ["신체활동", "음식섭취", "정신건강"],
#     "관계와 사회생활": ["공동체 (가족, 친구, 그 외 공동체)", "온라인 관계"],
#     "소비와 재정": ["소비 가치", "소비 습관"],
#     "스타일외모": ["패션뷰티 (의류)", "코스메틱뷰티 (피부,헤어)"],
#     "여가와 문화": ["여행", "디지털", "관람"],
#     "일상 습관": ["일상 습관", "환경", "경험(추억)", "일"]
# }

In [ ]:
# # 한글 변환 문장형 
# Label2_Ori = {  
#     "건강 웰빙": ["신체 움직임을 통한 건강 활동", "섭취를 통한 건강 활동", "정신적, 심적인 건강 활동"],
#     "관계와 사회생활": ["온라인상에서 형성된 관계", "오프라인에서 형성된 관계"],
#     "소비와 재정": ["소비를 통해 가치관을 표현", "소비를 통해 이득을 취하는 경우"],
#     "스타일 외모": ["패션 관련 뷰티", "패션 외적인 뷰티"],
#     "여가와 문화": ["온라인 기반 문화생활", "여행 기반 오프라인 문화생활", "여행 이외의 모든 오프라인 문화생활"],
#     "일상 요소": ["일상적으로 반복하는 행동", "환경과 관련된 행동", "경험 추억 등 과거와 관련된 행동", "학업, 직장 등 일 관련 행동"]
# }

In [ ]:
# 한글 변환 문장형 + 정확도

Label2 = {
    "건강 웰빙": [
        "신체 움직임을 통한 건강: 운동, 체력 관리, 수면 등 '활동' 자체.",
        "섭취를 통한 건강: 다이어트, 식단, 건강기능식품 등 '섭취' 관련.",
        "정신적, 심적인 건강: 스트레스, 감정 상태(걱정, 불편함), 명상 등 '심리'나 '상태'."
    ],
    "관계와 사회생활": [
        "온라인에서 형성된 관계: SNS, 커뮤니티, 온라인 게임 등 '사람 대 사람'의 온라인 교류. (AI 챗봇 같은 '도구' 사용은 제외)",
        "오프라인에서 형성된 관계: 가족, 친구, 직장 동료, 반려동물 등 현실의 관계."
    ],
    "소비와 재정": [
        "소비를 통해 가치관을 표현: 선물 선호, 물건 처분 방식, '나를 위한 소비' 등 개인의 '철학/가치'가 반영된 소비.",
        "소비를 통해 이득을 취하는 경우: 할인, 캐시백, 빠른 배송, OTT 개수, 월 지출액 등 '비용/혜택/효율' 중심의 소비."
    ],
    "스타일 외모": [
        "패션 관련 뷰티: 의류, 패션 아이템, 코디 같은 패션 성향.",
        "패션 외적인 뷰티: 피부 상태, 스킨케어 제품, 화장 등."
    ],
    "여가와 문화": [
        "온라인 기반 문화생활: OTT 시청, 웹툰, 웹소설, 음악 감상 등.",
        "여행 기반 오프라인 문화생활: 국내/해외 여행, 여행 계획, 여행 *방식* (예: 계획형/즉흥형 스타일).",
        "여행 이외의 모든 오프라인 문화생활: 물놀이, 전통시장 방문, 콘서트, 전시, 맛집 탐방 등."
    ],
    "일상 요소": [
        "일상적으로 반복하는 행동: 식습관(야식, 간식), 기상/알람 습관, 앱 사용 빈도 등 '루틴'이나 '습관'.",
        "환경과 관련된 행동: 날씨(비), 재활용, 환경보호, 이사 등 '외부 환경'에 대한 반응이나 행동.",
        "경험 추억 등 과거와 관련된 행동: 과거의 기억, 추억 (예: 초등학생 시절).",
        "학업, 직장 등 업무 관련 행동: 공부, 업무 방식 등."
    ],
    "기술 및 정보": [
        "디지털 도구 활용: AI 챗봇 사용, 정보 검색, 특정 앱의 '기능' 활용, 개인정보보호 설정 등."
    ]
}
    


In [ ]:
prompt_template = ChatPromptTemplate.from_messages(["""
[분류 규칙]
1. 반드시 [카테고리 정의]에 명시된 하위 카테고리 중 하나로만 분류해야 합니다.
2. 질문의 '의도'를 파악하고, 표면적인 '단어'에 속지 마세요.
3. 만약 정의된 어떤 하위 카테고리에도 속하지 않는다고 판단되면, 'main'과 'sub'에 "N/A"를 반환하세요.

[헷갈리는 항목 분류 예제]
- 질문: "요즘 어떤 분야에서 AI 서비스를 활용하고 계신가요?"
  - (분석: 'AI 서비스'는 '온라인 관계'가 아닌 '디지털 도구'임)
  - 분류: {{"main": "기술 및 정보", "sub": "디지털 도구 활용"}}

- 질문: "여행갈 때 어떤 스타일에 더 가까우신가요?"
  - (분석: '스타일'이라는 단어가 있지만, 문맥상 '여행 방식'을 의미함)
  - 분류: {{"main": "여가와 문화", "sub": "여행 기반 오프라인 문화생활"}}

- 질문: "다가오는 여름철 가장 걱정되는 점이 무엇인가요?"
  - (분석: '걱정'은 '반복 행동'이 아닌 '심리 상태'임)
  - 분류: {{"main": "건강 웰빙", "sub": "정신적, 심적인 건강"}}

- 질문: "평소 개인정보보호를 위해 어떤 습관이 있으신가요?"
  - (분석: '일상 습관'일 수 있으나, '개인정보보호'는 '디지털 도구' 설정과 더 관련 깊음)
  - 분류: {{"main": "기술 및 정보", "sub": "디지털 도구 활용"}}
반드시 JSON 형식으로만 반환하세요.
- main: 상위 카테고리 이름 (예: {main_cats})
- sub: 해당 상위 카테고리에 속하는 하위 카테고리
- score: 확신도 0.0~1.0 (소수점 두 자리)


카테고리와 하위 카테고리:
{cats_block}

출력 예시:
{{
  "main": "일상 요소",
  "sub": "환경과 관련된 행동",
  "score": 0.91
}}

다음 문장을 분류하세요:
"{input_text}"
"""])

[분류 규칙]
1. 반드시 [카테고리 정의]에 명시된 하위 카테고리 중 하나로만 분류해야 합니다.
2. 질문의 '의도'를 파악하고, 표면적인 '단어'에 속지 마세요.
3. 만약 정의된 어떤 하위 카테고리에도 속하지 않는다고 판단되면, 'main'과 'sub'에 "N/A"를 반환하세요.

[헷갈리는 항목 분류 예제]
- 질문: "요즘 어떤 분야에서 AI 서비스를 활용하고 계신가요?"
  - (분석: 'AI 서비스'는 '온라인 관계'가 아닌 '디지털 도구'임)
  - 분류: {"main": "기술 및 정보", "sub": "디지털 도구 활용"}

- 질문: "여행갈 때 어떤 스타일에 더 가까우신가요?"
  - (분석: '스타일'이라는 단어가 있지만, 문맥상 '여행 방식'을 의미함)
  - 분류: {"main": "여가와 문화", "sub": "여행 기반 오프라인 문화생활"}

- 질문: "다가오는 여름철 가장 걱정되는 점이 무엇인가요?"
  - (분석: '걱정'은 '반복 행동'이 아닌 '심리 상태'임)
  - 분류: {"main": "건강 웰빙", "sub": "정신적, 심적인 건강"}

- 질문: "평소 개인정보보호를 위해 어떤 습관이 있으신가요?"
  - (분석: '일상 습관'일 수 있으나, '개인정보보호'는 '디지털 도구' 설정과 더 관련 깊음)
  - 분류: {"main": "기술 및 정보", "sub": "디지털 도구 활용"}

In [ ]:
main_cats = ", ".join(Label2.keys())
cats_block = "\n".join([f"{main}: {', '.join(subs)}" for main, subs in Label2.items()])

PROMPT = prompt_template

chain = PROMPT | llm | StrOutputParser()


In [23]:
def classify_texts(texts: List[str]) -> List[Dict]:
    results = []
    for t in texts:
        try:
            resp = chain.invoke({
                "input_text": t,
                "main_cats": main_cats,
                "cats_block": cats_block
            }).strip()
            
            start = resp.find('{')
            end = resp.rfind('}') + 1
            parsed = json.loads(resp[start:end])
            main = parsed.get("main")
            sub = parsed.get("sub")
            score = float(parsed.get("score", 0.0))
            
            results.append({
                "text": t,
                "main": main,
                "sub": sub,
                "score": round(score, 2),
                "raw": resp
            })
        except Exception as e:
            results.append({
                "text": t,
                "main": None,
                "sub": None,
                "score": 0.0,
                "raw": resp if 'resp' in locals() else "",
                "error": str(e)
            })
    return results


In [ ]:
if __name__ == "__main__":
    sample_texts = [
    #생략
    ]

    outputs = classify_texts(sample_texts)
    print(json.dumps(outputs, ensure_ascii=False, indent=2))